# **Import libraries**

In [1]:
# keras imports for the dataset and building our neural network
import keras 
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Input,BatchNormalization
from keras.utils import np_utils
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')

# **Section I: Used datasets**

# **Load and preprocess the CIFAR10 dataset**

In [2]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
print('Training data shape : ', X_train.shape, y_train.shape)

print('Testing data shape : ', X_test.shape, y_test.shape)

Training data shape :  (50000, 32, 32, 3) (50000, 1)
Testing data shape :  (10000, 32, 32, 3) (10000, 1)


In [4]:
#make a input vector so we 
#reshape it into input format for training and testing sets. After that change all datatypes into floats

# building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [5]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [6]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", y_train.shape)

Shape before one-hot encoding:  (50000, 1)
Shape after one-hot encoding:  (50000, 10)


In [7]:
print(X_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


# **Section II:**

# **VGG16 Model work**

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape =(32,32,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Adding a Dense layer with 256 neurons
x = Dense(256, activation = 'relu')(x)

# Add a DropOut layer with Drop out ratio of 20%
x =Dropout(0.2)(x)

# Add a Batch Normalization layer
x = BatchNormalization(axis=-1)(x)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a DropOut layer with Drop out ratio of 20%
x =Dropout(0.2)(x)

# Add a final softmax layer for classification output
x = layers.Dense(10, activation='softmax')(x)

model=Model(base_model.input, x)

model.compile(optimizer =Adam(learning_rate=0.001), loss ='categorical_crossentropy',metrics = ['acc'])

In [11]:
history=model.fit(X_train, y_train, 
                  batch_size=64, 
                  epochs=40, 
                  validation_data=(X_test, y_test))

Epoch 1/40
782/782 [==============================] - 23s 16ms/step - loss: 1.4454 - acc: 0.4924 - val_loss: 1.2732 - val_acc: 0.5470
Epoch 2/40
782/782 [==============================] - 11s 14ms/step - loss: 1.2632 - acc: 0.5564 - val_loss: 1.2316 - val_acc: 0.5646
Epoch 3/40
782/782 [==============================] - 12s 15ms/step - loss: 1.2068 - acc: 0.5761 - val_loss: 1.2227 - val_acc: 0.5765
Epoch 4/40
782/782 [==============================] - 11s 14ms/step - loss: 1.1757 - acc: 0.5890 - val_loss: 1.1600 - val_acc: 0.5923
Epoch 5/40
782/782 [==============================] - 12s 15ms/step - loss: 1.1416 - acc: 0.5978 - val_loss: 1.1554 - val_acc: 0.5959
Epoch 6/40
782/782 [==============================] - 12s 15ms/step - loss: 1.1213 - acc: 0.6071 - val_loss: 1.1543 - val_acc: 0.5952
Epoch 7/40
782/782 [==============================] - 11s 14ms/step - loss: 1.1106 - acc: 0.6115 - val_loss: 1.1525 - val_acc: 0.5993
Epoch 8/40
782/782 [==============================] - 12s 16ms

In [12]:
# record the training and validation accuracy and loss after each epoch
train_acc = history.history['acc']
train_loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

# save the training log in a file
with open('Model_report.txt', 'w') as f:
    for i in range(len(train_acc)):
        f.write("Epoch {}: train_acc = {}, train_loss = {}, val_acc = {}, val_loss = {}\n".format(
            i+1, train_acc[i], train_loss[i], val_acc[i], val_loss[i]))

# **Section III:**

In [13]:
# from keras.applications.vgg16 import preprocess_input
import numpy as np

def extract_deep_features(x):
  # x = preprocess_input(x)
  features = model.predict(x)
  features = np.reshape(features, (features.shape[0], -1))
  return features

x_train_features = extract_deep_features(X_train)
x_test_features = extract_deep_features(X_test)

313/313 [==============================] - 2s 7ms/step


In [14]:
x_train_features.shape

(50000, 10)

In [15]:
x_test_features.shape

(10000, 10)

In [16]:
# Convert one-hot encoded labels back to integer labels
y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

# **Training SVM**

In [17]:
##     Training SVM model on the extracted Features 

from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train_features, y_train_int)

score = clf.score(x_test_features, y_test_int)
print('Test accuracy:', score)

Test accuracy: 0.6197


In [18]:
score = clf.score(x_train_features, y_train_int)
print('Test accuracy:', score)

Test accuracy: 0.79872


# **Train custom model using the deep features**

In [19]:
# Define input shape
input_shape = (x_train_features.shape[1],)

# Define the input layer
inputs = Input(shape=input_shape)

# Define the first fully connected layer
fc1 = Dense(units=256, activation='relu', name='fc1')(inputs)

# Add a dropout layer to prevent overfitting
dropout1 = Dropout(rate=0.4, name='dropout1')(fc1)

# Define the second fully connected layer
fc2 = Dense(units=128, activation='relu', name='fc2')(dropout1)

# Add a dropout layer to prevent overfitting
dropout2 = Dropout(rate=0.3, name='dropout2')(fc2)

# Define the output layer
outputs = Dense(units=10, activation='softmax', name='output')(dropout2)

# Create the model
model = Model(inputs=inputs, outputs=outputs, name='my_custom_model')

# Compile the model with categorical cross-entropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Train the model on the extracted features
history = model.fit(x_train_features, y_train, batch_size=64, epochs=30, validation_data=(x_test_features, y_test))


Epoch 1/30
782/782 [==============================] - 4s 4ms/step - loss: 0.7442 - accuracy: 0.7801 - val_loss: 1.3434 - val_accuracy: 0.6195
Epoch 2/30
782/782 [==============================] - 3s 4ms/step - loss: 0.6421 - accuracy: 0.7947 - val_loss: 1.3328 - val_accuracy: 0.6190
Epoch 3/30
782/782 [==============================] - 4s 5ms/step - loss: 0.6273 - accuracy: 0.7956 - val_loss: 1.3319 - val_accuracy: 0.6186
Epoch 4/30
782/782 [==============================] - 3s 4ms/step - loss: 0.6159 - accuracy: 0.7957 - val_loss: 1.3104 - val_accuracy: 0.6179
Epoch 5/30
782/782 [==============================] - 3s 4ms/step - loss: 0.6080 - accuracy: 0.7951 - val_loss: 1.3124 - val_accuracy: 0.6180
Epoch 6/30
782/782 [==============================] - 3s 4ms/step - loss: 0.6030 - accuracy: 0.7947 - val_loss: 1.3456 - val_accuracy: 0.6178
Epoch 7/30
782/782 [==============================] - 4s 5ms/step - loss: 0.5995 - accuracy: 0.7954 - val_loss: 1.3492 - val_accuracy: 0.6190
Epoch 